In [1]:
# clearing datas -> filter columns, see relation between data, replace missing values.
# model training -> do preprocessing data,make crossValidation, do Fit, save the result trained
# model test -> use the model and check accuracy

import pandas as pd

_data_train = pd.read_csv("/home/DS/20200329_DS_codenationENEM/1.Original/train.csv")
_data_test = pd.read_csv("/home/DS/20200329_DS_codenationENEM/1.Original/test.csv")

In [2]:
#list of all columns we want
ml_features = [
    ""
    "NU_NOTA_CN",
    "NU_NOTA_CH",
    "NU_NOTA_LC",
    "NU_NOTA_COMP1",
    "NU_NOTA_COMP2",
    "NU_NOTA_COMP3",
    "NU_NOTA_COMP4",
    "NU_NOTA_COMP5",
    "NU_NOTA_REDACAO"
]

In [3]:
#prepare data that gonna use

import matplotlib.pyplot as plt
import seaborn as sns

#preparing data
dataTrain = _data_train.loc[:, (ml_features + ["NU_NOTA_MT"])]
dataTest = _data_test.loc[:, ml_features]

dataIncritionWithoutNull = _data_test.loc[:, ml_features + ["NU_INSCRICAO"]]

print(f"LENGTH NULL DATATRAIN :\n {dataTrain.isnull().sum()}\n")
print(f"LENGTH NULL DATATEST :\n {dataTest.isnull().sum()}\n")
print(f"LENGTH NULL DATAINCRITIONWITHOUTNULL :\n {dataIncritionWithoutNull.isnull().sum()}\n")
print(f"BEFORE HANDLE WITH NULL AND NAN\n dataTest shape == {dataTest.shape}\t dataTrain shape == {dataTrain.shape}\t dataIncritionWithoutNull == {dataIncritionWithoutNull.shape}")

LENGTH NULL DATATRAIN :
 NU_NOTA_CN         3389
NU_NOTA_CH         3389
NU_NOTA_LC         3597
NU_NOTA_COMP1      3597
NU_NOTA_COMP2      3597
NU_NOTA_COMP3      3597
NU_NOTA_COMP4      3597
NU_NOTA_COMP5      3597
NU_NOTA_REDACAO    3597
NU_NOTA_MT         3597
dtype: int64

LENGTH NULL DATATEST :
 NU_NOTA_CN         1134
NU_NOTA_CH         1134
NU_NOTA_LC         1199
NU_NOTA_COMP1      1199
NU_NOTA_COMP2      1199
NU_NOTA_COMP3      1199
NU_NOTA_COMP4      1199
NU_NOTA_COMP5      1199
NU_NOTA_REDACAO    1199
dtype: int64

LENGTH NULL DATAINCRITIONWITHOUTNULL :
 NU_NOTA_CN         1134
NU_NOTA_CH         1134
NU_NOTA_LC         1199
NU_NOTA_COMP1      1199
NU_NOTA_COMP2      1199
NU_NOTA_COMP3      1199
NU_NOTA_COMP4      1199
NU_NOTA_COMP5      1199
NU_NOTA_REDACAO    1199
NU_INSCRICAO          0
dtype: int64

BEFORE HANDLE WITH NULL AND NAN
 dataTest shape == (4576, 9)	 dataTrain shape == (13730, 10)	 dataIncritionWithoutNull == (4576, 10)


In [4]:
#handling and null and nan
dataTrain.dropna(axis=0, inplace=True)
dataTest.dropna(axis=0, inplace=True)

dataIncritionWithoutNull.dropna(axis=0, inplace=True)
dataIncritionWithoutNull.drop(columns=ml_features, axis=1, inplace=True)

#y = target
target = dataTrain.loc[:,"NU_NOTA_MT"]

#erasing of dataTrain variable our column NU_NOTA_MT that doesn't more necessary
dataTrain.drop(["NU_NOTA_MT"], axis=1, inplace=True)

print(f"LENGTH NULL DATATRAIN :\n {dataTrain.isnull().sum()}\n")
print(f"LENGTH NULL DATATEST :\n {dataTest.isnull().sum()}\n")
print(f"BEFORE HANDLE WITH NULL AND NAN\n dataTest shape == {dataTest.shape}\t dataTrain shape == {dataTrain.shape}\t dataIncritionWithoutNull == {dataIncritionWithoutNull.shape}")

LENGTH NULL DATATRAIN :
 NU_NOTA_CN         0
NU_NOTA_CH         0
NU_NOTA_LC         0
NU_NOTA_COMP1      0
NU_NOTA_COMP2      0
NU_NOTA_COMP3      0
NU_NOTA_COMP4      0
NU_NOTA_COMP5      0
NU_NOTA_REDACAO    0
dtype: int64

LENGTH NULL DATATEST :
 NU_NOTA_CN         0
NU_NOTA_CH         0
NU_NOTA_LC         0
NU_NOTA_COMP1      0
NU_NOTA_COMP2      0
NU_NOTA_COMP3      0
NU_NOTA_COMP4      0
NU_NOTA_COMP5      0
NU_NOTA_REDACAO    0
dtype: int64

BEFORE HANDLE WITH NULL AND NAN
 dataTest shape == (3360, 9)	 dataTrain shape == (10097, 9)	 dataIncritionWithoutNull == (3360, 1)


In [5]:
#separing tests train
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(dataTrain, target, test_size=0.30, random_state = 0)

In [6]:
#ML LightGBM
import lightgbm as lgb

params = {
    "objective" : "regression",
    "metric" : "rmse",
    "n_estimators": 1000,
    "num_leaves" : 30,
    "min_child_samples" : 30,
    "learning_rate" : 0.005,
    "verbosity" : -1
}

#creating dataset for lgb
lgb_dataFrameTrain = lgb.Dataset(X_train, label=Y_train)
lgb_dataFrameValidation = lgb.Dataset(X_validation, label=Y_validation)

ml = lgb.train(params, lgb_dataFrameTrain, 8000, valid_sets=[lgb_dataFrameValidation], early_stopping_rounds=100, verbose_eval=100)

Training until validation scores don't improve for 100 rounds


/home/xx/.local/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	valid_0's rmse: 85.5342
[200]	valid_0's rmse: 79.7755
[300]	valid_0's rmse: 77.6044
[400]	valid_0's rmse: 76.872
[500]	valid_0's rmse: 76.66
[600]	valid_0's rmse: 76.6166
Early stopping, best iteration is:
[588]	valid_0's rmse: 76.6078


In [9]:
#creating output
import numpy as np

#return with predicts data worked not null
dataOutput = dataIncritionWithoutNull.copy()
dataOutput["NU_NOTA_MT"] = np.around(ml.predict(dataTest), 2)

dataOutput = pd.concat([dataOutput, dataTest], axis=1, join="outer")

#concat the nulls not worked

dataOutput.drop(columns= ml_features + ["NU_INSCRICAO"], axis=1, inplace=True)

dataInscritonNull = _data_test.loc[:, "NU_INSCRICAO"].to_frame()
dataOutput = pd.concat([dataInscritonNull, dataOutput], axis=1, join="outer")

dataOutput.to_csv("/home/DS/20200329_DS_codenationENEM/3.Deployed/codenationENEM_done.csv")
dataOutput

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,434.92
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,454.03
2,b38a03232f43b11c9d0788abaf060f7366053b6d,591.37
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,NaN
4,715494628a50142ce8cb17191cfe6d0f3cae0934,532.26
...,...,...
4571,dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,454.26
4572,a75fa8770257e7c9368d059fe53d9ef431f4bdef,477.23
4573,655fa6306720ff16e825903b5422a46608a77545,693.43
4574,1f4bc3e3d56212d500625bfe8ac78ccff4362293,455.27
